# Epsilon Fund — Strategy Testing
---

In [ ]:
import pandas as pd
import numpy as np
import sys

# ── Set your repo root path ────────────────────────────────────────────────────
ROOT = r'C:\Users\alvar\OneDrive\Escritorio\Projects\Epsilon Fund\Epsilon-Quant-Research'  # ← change to yours
# ──────────────────────────────────────────────────────────────────────────────

sys.path.append(ROOT + r'\infrastructure\data')
sys.path.append(ROOT + r'\infrastructure\backtester')

from binance_client import get_binance_client, get_data, get_multiple_data
from engine import backtest

---
## Data

**Pairs** — any Binance pair in `BASEQUOTE` format (e.g. `BTCUSDT`, `ETHUSDT`, `SOLUSDT`, `BNBUSDT`).  
Verify availability at [binance.com/en/trade](https://www.binance.com/en/trade).

**Intervals** — `'1m'` `'5m'` `'15m'` `'1h'` `'4h'` `'1d'` `'1w'`

**Lookback** — days of history: `365` (1y) · `730` (2y) · `1825` (5y) · `2555` (7y, recommended minimum)

In [ ]:
SYMBOL   = 'BTCUSDT'
INTERVAL = '1d'
LOOKBACK = 2555

# ── Multiple pairs (optional) ──────────────────────────────────────────────────
# SYMBOLS = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT']
# data_dict = get_multiple_data(client, SYMBOLS, INTERVAL, LOOKBACK)
# Access via: data_dict['BTCUSDT_1d'], data_dict['ETHUSDT_1d'] ...
# ──────────────────────────────────────────────────────────────────────────────

client = get_binance_client()
df = get_data(client, SYMBOL, INTERVAL, LOOKBACK)

print(f'{SYMBOL} | {INTERVAL} | {df.index[0].date()} -> {df.index[-1].date()} | {len(df)} rows')
df.tail(3)

---
## Strategy

**Available columns:** `Open` `High` `Low` `Close` `Volume`

**Required output:** a `position` column — `1` long · `0` flat · `-1` short  
**Optional:** `position_size` column (0–1) to use fractional capital

> Signals are shifted 1 bar by the engine — no need to shift manually.

In [ ]:
strategy_df = df.copy()

# ── Strategy logic ─────────────────────────────────────────────────────────────


# ──────────────────────────────────────────────────────────────────────────────

strategy_df.dropna(inplace=True)
strategy_df['position'].value_counts()

---
## Backtest

| Parameter | Options | Default |
|---|---|---|
| `cost` | Cost per trade as decimal — `0.001` = 0.1% | `0.0` |
| `show_plot` | `True` / `False` | `True` |
| `save_html` | Filename string or `None` | `None` |
| `show_trades` | Overlay entry/exit markers on price chart | `False` |
| `benchmark_data` | DataFrame with `Close` column for buy & hold comparison | same asset |

In [ ]:
results = backtest(
    data           = strategy_df,
    cost           = 0.0,
    show_plot      = True,
    save_html      = None,
    show_trades    = False,
    benchmark_data = None
)

print(f"Return        {results['total_return']*100:>8.2f}%")
print(f"Sharpe        {results['sharpe_ratio']:>8.2f}")
print(f"Max Drawdown  {results['max_drawdown']*100:>8.2f}%")
print(f"Profit Factor {results['profit_factor']:>8.2f}")
print(f"Win Rate      {results['win_rate']*100:>8.2f}%")
print(f"# Trades      {results['num_trades']:>8}")